# Inference API

> Provides high-level functions for loading a trained Indic-CLIP model and performing inference (feature extraction, similarity computation).

In [ ]:
#| default_exp inference

## Colab Setup

In [1]:
#| hide

from pathlib import Path
import sys
import os

project_parent = '/workspace'
if Path('/workspace/indic-clip').exists():
    project_parent = '/workspace/indic-clip'
    if project_parent not in sys.path:
        sys.path.insert(0, project_parent)
        print(f"Added {project_parent} to sys.path")
    try:
        import indic_clip.core
        print("Imported indic_clip.core after path adjustment.")
    except ModuleNotFoundError:
        print("ERROR: Still cannot find indic_clip.core. Ensure project structure is correct.")
        print("Expected: /workspace/indic-clip/indic-clip/core.py or similar in Drive")          

Added /workspace/indic-clip to sys.path
Imported indic_clip.core after path adjustment.


In [2]:
#| hide
# Install requirements if needed (especially in Colab)
# !pip install -qr requirements.txt

In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
#| export
import torch
import albumentations
import logging
import PIL
from pathlib import Path
from PIL import Image
from typing import List, Union, Dict, Optional

from fastcore.all import *
from fastai.vision.all import *
from torchvision.transforms import Compose
import torchvision.transforms as T

# --- Project Imports ---
from indic_clip.core import (
        get_logger, setup_logging, DEFAULT_IMAGE_SIZE, DEFAULT_EMBED_DIM, PRETRAINED_TOKENIZER_NAME, TOKENIZER_PATH,
        CHECKPOINT_PATH, ensure_dir, PROJECT_ROOT
    )
from indic_clip.model.clip import IndicCLIP
from indic_clip.data.tokenization import IndicBERTTokenizer

logger = get_logger(__name__)

Reloaded indic_clip.core
Reloaded indic_clip.core
Reloaded indic_clip.core
Reloaded indic_clip.core


## Core Inference Functions

In [12]:
#| export
def load_indic_clip_model(
    checkpoint_path: Union[str, Path],
    model_config: Optional[Dict] = None, # Optional: pass model config if not saved with ckpt
    device: Optional[Union[str, torch.device]] = None
) -> IndicCLIP:
    """
    Loads a trained IndicCLIP model from a checkpoint file.

    Args:
        checkpoint_path (Union[str, Path]): Path to the .pth checkpoint file.
        model_config (Optional[Dict]): Dictionary containing model configuration args
                                      (embed_dim, vision_model_name, text_model_name, etc.)
                                      needed to instantiate the model structure.
                                      If None, attempts to infer from defaults or common patterns.
        device (Optional[Union[str, torch.device]]): Device to load the model onto ('cuda', 'cpu', etc.).
                                                      Defaults to CUDA if available, else CPU.

    Returns:
        IndicCLIP: The loaded model instance in evaluation mode.

    Raises:
        FileNotFoundError: If the checkpoint file does not exist.
        TypeError: If model_config is needed but not provided.
        RuntimeError: If there's an issue loading the state dict.
    """
    checkpoint_path = Path(checkpoint_path)
    if not checkpoint_path.is_file():
        raise FileNotFoundError(f"Checkpoint file not found: {checkpoint_path}")

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Loading model onto device: {device}")

    # --- Instantiate Model Structure ---
    # Best practice: Save model config with checkpoint. If not, require it here.
    if model_config is None:
        # Attempt to use defaults - this might be incorrect!
        logger.warning("Model config not provided. Attempting to load using default parameters. This may fail or lead to unexpected behavior if the checkpoint was saved with different settings.")
        model_config = {
            'embed_dim': DEFAULT_EMBED_DIM,
            'vision_model_name': 'vit_base_patch16_224', # Make sure this matches your training!
            'vision_pretrained': False, # Pretrained flag doesn't matter much for loading weights
            'text_model_name': PRETRAINED_TOKENIZER_NAME, # Make sure this matches!
            'text_pretrained': False,
            # Tokenizer instance needed if custom tokens were added, load separately!
            'tokenizer': IndicBERTTokenizer.load_tokenizer(TOKENIZER_PATH) # Assume default path
        }
        # If tokenizer is required and fails to load, raise an error
        if not model_config['tokenizer']:
             raise ValueError("Could not load default tokenizer required for model instantiation.")

    # Ensure tokenizer is present in config for IndicCLIP constructor
    if 'tokenizer' not in model_config or model_config['tokenizer'] is None:
         logger.warning("Loading model without providing a tokenizer instance. Trying to load default.")
         try:
             model_config['tokenizer'] = IndicBERTTokenizer.load_tokenizer(TOKENIZER_PATH)
             if not model_config['tokenizer']:
                 raise ValueError("Failed to load default tokenizer.")
         except Exception as e:
             raise ValueError(f"Could not load default tokenizer required for model instantiation from {TOKENIZER_PATH}. Please provide it in model_config. Error: {e}")

    try:
        model = IndicCLIP(**model_config)
        logger.info(f"Instantiated IndicCLIP model structure with config: {model_config}")
    except Exception as e:
        logger.error(f"Failed to instantiate model with provided config: {e}", exc_info=True)
        raise TypeError(f"Could not instantiate IndicCLIP model. Check model_config. Error: {e}")

    # --- Load State Dict ---
    try:
        # Map location ensures model loads correctly regardless of where it was saved
        # *** ADD weights_only=False HERE ***
        state_dict = torch.load(checkpoint_path, map_location='cpu', weights_only=False)
        logger.info(f"Loaded checkpoint using weights_only=False.")

        # Handle potential keys mismatch (e.g., if saved directly or via Learner.save)
        if 'model' in state_dict:
            state_dict = state_dict['model']
        elif 'state_dict' in state_dict:
            state_dict = state_dict['state_dict']
        # If neither 'model' nor 'state_dict' key is present, assume the whole dict is the state_dict
        
        # Remove potential DDP prefix 'module.'
        state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}

        # Load the weights
        model.load_state_dict(state_dict)
        logger.info(f"Successfully loaded model weights from {checkpoint_path}")

    except Exception as e:
        logger.error(f"Error loading state dict from {checkpoint_path}: {e}", exc_info=True)
        # Add the original exception 'e' to the message for more context
        raise RuntimeError(f"Failed to load state dict. Ensure checkpoint is valid and config matches. Error: {e}")

    model.eval() # Set model to evaluation mode
    model.to(device) # Move model to the target device
    logger.info("Model set to evaluation mode.")

    return model

In [13]:
#| export
def _get_image_transform(img_size: int = DEFAULT_IMAGE_SIZE):
    """Creates a standard image transform pipeline for inference."""
    # Use transforms similar to validation set
    return Compose([
        Resize(img_size, method='squish'), # Or 'pad'
        ToTensor()
    ])

In [14]:
#| export
@torch.no_grad()
def extract_image_features(
    model: IndicCLIP,
    image_input: Union[str, Path, Image.Image, torch.Tensor, List[Union[str, Path, Image.Image, torch.Tensor]]],
    img_size: int = DEFAULT_IMAGE_SIZE,
    batch_size: int = 32,
    device: Optional[Union[str, torch.device]] = None
) -> torch.Tensor:
    """
    Extracts normalized features for one or more images using the IndicCLIP model.
    Manually handles image loading, resizing, tensor conversion, and normalization
    to ensure device consistency.

    Args:
        model (IndicCLIP): The loaded IndicCLIP model instance.
        image_input: A single image (path, PIL Image, or preprocessed Tensor) or a list of images.
        img_size (int): The image size the model expects (both width and height).
        batch_size (int): Batch size for processing multiple images.
        device (Optional[Union[str, torch.device]]): Device to use for inference. If None, uses model's device.

    Returns:
        torch.Tensor: A tensor containing the normalized image features (N, embed_dim).
    """
    model.eval()
    if device is None:
        device = next(model.parameters()).device
    else:
        model.to(device) # Ensure model is on the specified device

    # Prepare normalization transform (will be applied on device later)
    # Using torchvision's Normalize
    mean_tensor = torch.tensor(imagenet_stats[0], device=device)
    std_tensor = torch.tensor(imagenet_stats[1], device=device)
    
    def normalize_batch(b):
        # b shape is (N, C, H, W)
        b = b.to(device) # Ensure batch is on the correct device
        mean = mean_tensor.view(1, -1, 1, 1) # Reshape for broadcasting
        std = std_tensor.view(1, -1, 1, 1)
        return (b - mean) / std

    if not isinstance(image_input, list):
        image_input = [image_input]

    all_features = []
    logger.info(f"Processing {len(image_input)} images in batches of {batch_size}...")
    for i in range(0, len(image_input), batch_size):
        batch_inputs = image_input[i:i+batch_size]
        processed_batch_cpu = [] # Collect processed tensors on CPU first
        
        for item in batch_inputs:
            try:
                if isinstance(item, (str, Path)):
                    # 1. Load PIL image
                    img_pil = Image.open(item).convert('RGB')
                elif isinstance(item, Image.Image):
                    # 1. Assume PIL Image
                    img_pil = item.convert('RGB')
                elif isinstance(item, torch.Tensor):
                    # If it's already a tensor, assume it's *almost* ready (e.g., NCHW, float)
                    # We'll just move it to CPU before potential stacking and handle normalization later
                    processed_batch_cpu.append(item.cpu())
                    continue # Skip manual processing for tensors
                else:
                    logger.warning(f"Unsupported image input type: {type(item)}. Skipping.")
                    continue

                # --- Manual Processing for PIL images ---
                # 2. Resize PIL image
                # Use ANTIALIAS for better quality resizing
                img_pil = img_pil.resize((img_size, img_size), Image.Resampling.LANCZOS) 
                
                # 3. Convert PIL image to PyTorch Tensor (on CPU)
                # This converts HWC [0, 255] uint8 to CHW [0.0, 1.0] float32
                img_tensor_cpu = T.functional.to_tensor(img_pil)
                
                processed_batch_cpu.append(img_tensor_cpu)
                # --- End Manual Processing ---

            except Exception as e:
                 logger.error(f"Failed to process image item: {item}. Error: {e}", exc_info=True)
                 # Continue to next item in batch if one fails

        if not processed_batch_cpu:
            logger.warning(f"Batch {i//batch_size + 1} is empty after processing/errors.")
            continue # Skip if batch is empty

        # Stack processed images into a batch tensor (on CPU)
        try:
            batch_tensor_cpu = torch.stack(processed_batch_cpu)
        except RuntimeError as e:
            logger.error(f"Error stacking tensors in batch {i//batch_size + 1}. Check image processing consistency. Error: {e}")
            for idx, t in enumerate(processed_batch_cpu): logger.error(f"  Item {idx} shape: {t.shape}, dtype: {t.dtype}")
            continue # Skip this batch

        # Move the batch tensor to the target device
        batch_tensor = batch_tensor_cpu.to(device)

        # Normalize the batch *on the target device* using explicit normalization
        try:
             batch_tensor = normalize_batch(batch_tensor) # Apply normalization
        except Exception as e:
             logger.error(f"Error normalizing batch {i//batch_size + 1} on device {device}. Error: {e}")
             continue # Skip batch if normalization fails

        # --- Extract features using the model ---
        try:
            features = model.encode_image(batch_tensor) # Model expects input on its device
            all_features.append(features.cpu()) # Move features back to CPU immediately to free GPU mem
        except Exception as e:
            logger.error(f"Error during model.encode_image for batch {i//batch_size + 1}. Error: {e}", exc_info=True)
            # Optionally log input tensor stats
            logger.error(f"Input batch tensor stats: mean={batch_tensor.mean()}, std={batch_tensor.std()}, min={batch_tensor.min()}, max={batch_tensor.max()}")
            continue # Skip batch if encoding fails

    if not all_features:
        logger.error("No image features could be extracted.")
        # Return an empty tensor with the correct embedding dimension
        try:
            # Accessing projection layer directly might be fragile if model structure changes
            # It's better if embed_dim is stored on the model, e.g., self.embed_dim
            embed_dim = model.visual_projection.out_features
        except AttributeError:
            logger.warning("Could not determine model embedding dimension from visual_projection, using default.")
            embed_dim = DEFAULT_EMBED_DIM # Fallback
        return torch.empty((0, embed_dim))

    logger.info(f"Successfully extracted features for {len(torch.cat(all_features))} images.")
    return torch.cat(all_features)

In [15]:
#| export
@torch.no_grad()
def extract_text_features(
    model: IndicCLIP,
    tokenizer: IndicBERTTokenizer,
    text_input: Union[str, List[str]],
    batch_size: int = 32,
    device: Optional[Union[str, torch.device]] = None
) -> torch.Tensor:
    """
    Extracts normalized features for one or more text strings using the IndicCLIP model.

    Args:
        model (IndicCLIP): The loaded IndicCLIP model instance.
        tokenizer (IndicBERTTokenizer): The tokenizer instance compatible with the model.
        text_input (Union[str, List[str]]): A single text string or a list of strings.
        batch_size (int): Batch size for processing multiple texts.
        device (Optional[Union[str, torch.device]]): Device to use for inference. If None, uses model's device.

    Returns:
        torch.Tensor: A tensor containing the normalized text features (N, embed_dim).
    """
    model.eval()
    if device is None:
        device = next(model.parameters()).device
    else:
        model.to(device)

    if isinstance(text_input, str):
        text_input = [text_input]

    all_features = []
    for i in range(0, len(text_input), batch_size):
        batch_texts = text_input[i:i+batch_size]
        
        try:
            # Tokenize the batch
            inputs = tokenizer.tokenize(batch_texts)
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)
            
            # Extract features
            features = model.encode_text(input_ids=input_ids, attention_mask=attention_mask)
            all_features.append(features.cpu()) # Move features to CPU
        except Exception as e:
            logger.error(f"Error processing text batch starting with '{batch_texts[0][:50]}...': {e}")
            continue # Skip batch on error

    if not all_features:
        logger.error("No text features could be extracted.")
        embed_dim = model.text_projection.out_features # Get embed_dim from model
        return torch.empty((0, embed_dim))

    return torch.cat(all_features)

In [16]:
#| export
@torch.no_grad()
def compute_similarity(
    model: IndicCLIP,
    image_features: torch.Tensor,
    text_features: torch.Tensor
) -> torch.Tensor:
    """
    Computes the similarity matrix between image and text features using the model's logit scale.

    Args:
        model (IndicCLIP): The loaded IndicCLIP model instance (used for logit_scale).
        image_features (torch.Tensor): Normalized image features (N, embed_dim).
        text_features (torch.Tensor): Normalized text features (M, embed_dim).

    Returns:
        torch.Tensor: A similarity matrix of shape (N, M).
    """
    model.eval()
    device = next(model.parameters()).device # Use model's device
    
    # Ensure features are on the correct device
    image_features = image_features.to(device)
    text_features = text_features.to(device)
    
    # Get the exponentiated logit scale from the model
    logit_scale = model.logit_scale.exp()
    
    # Compute similarity (dot product of normalized features, scaled by temperature)
    # Features should already be normalized by the extraction functions
    similarity = logit_scale * image_features @ text_features.t()
    
    return similarity

## Example Usage

In [22]:
#| eval: false 
# --- Example: Load Model and Run Inference ---
if __name__ == '__main__':
    print("--- Running Inference API Example ---")
    
    # --- Configuration ---
    # !! IMPORTANT: Replace with the *actual* path to your trained checkpoint !!
    # checkpoint_name = 'best_recall_interactive.pth' # From 10_training example
    checkpoint_name = 'best_valid_loss.pth' # Default name if SaveModelCallback wasn't configured fully
    print(f'CHECKPOINT_PATH: {CHECKPOINT_PATH}')
    checkpoint_file = CHECKPOINT_PATH / checkpoint_name
    
    # !! IMPORTANT: Provide the config used during training if it wasn't saved with the checkpoint !!
    #             (If Learner.save was used, config is often not saved)
    # Example: Use the smaller ResNet18 config from the 10_training notebook test
    model_configuration = {
        'embed_dim': 512, # Example: ResNet18 from test
        'vision_model_name': 'resnet50', # Example: ResNet18 from test
        'vision_pretrained': False, # Doesn't matter for loading state_dict
        'text_model_name': PRETRAINED_TOKENIZER_NAME,
        'text_pretrained': False,
        'tokenizer': IndicBERTTokenizer.load_tokenizer(TOKENIZER_PATH) # Load the tokenizer used for training
    }
    
    # Create dummy checkpoint if it doesn't exist for demonstration
    if not checkpoint_file.exists():
        print(f"Warning: Checkpoint {checkpoint_file} not found. Creating a dummy model and saving it.")
        ensure_dir(checkpoint_file.parent)
        dummy_model = IndicCLIP(**model_configuration) 
        torch.save(dummy_model.state_dict(), checkpoint_file)
        print("Dummy checkpoint created.")

    # --- Load Model and Tokenizer ---
    try:
        loaded_model = load_indic_clip_model(checkpoint_file, model_config=model_configuration)
        loaded_tokenizer = IndicBERTTokenizer.load_tokenizer(TOKENIZER_PATH)
        print("Model and tokenizer loaded successfully.")

        # --- Prepare Sample Data ---
        # Use a real image path if available, otherwise create dummy
        sample_image_path = PROJECT_ROOT / 'cat.jpg' # Create this image or use a real one
        if not sample_image_path.exists():
             print(f"Creating dummy image at: {sample_image_path}")
             dummy_pil_img = Image.new('RGB', (250, 300), color='cyan')
             dummy_pil_img.save(sample_image_path)

        sample_texts = [
            "एक बिल्ली सोफे पर सो रही है।", # A cat sleeping on a sofa
            "समुद्र तट पर सूर्यास्त।",    # Sunset on a beach
            "साड़ी पहने एक महिला।"       # A woman wearing a saree
        ]

        # --- Extract Features ---
        print("\nExtracting image features...")
        image_features = extract_image_features(loaded_model, sample_image_path)
        print(f"Image features shape: {image_features.shape}")

        print("\nExtracting text features...")
        text_features = extract_text_features(loaded_model, loaded_tokenizer, sample_texts)
        print(f"Text features shape: {text_features.shape}")

        # --- Compute Similarity ---
        print("\nComputing similarity...")
        similarity_matrix = compute_similarity(loaded_model, image_features, text_features)
        print(f"Similarity matrix shape: {similarity_matrix.shape}")
        print("Similarity scores:")
        print(similarity_matrix.cpu().numpy())

        # Find best matching text for the image
        best_match_idx = similarity_matrix.argmax().item()
        print(f"\nBest matching text for the image: '{sample_texts[best_match_idx]}' (Score: {similarity_matrix[0, best_match_idx]:.4f})")

    except FileNotFoundError as e:
        print(f"\nError: {e}")
        print("Please ensure the correct checkpoint and tokenizer paths are set.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
        
    # Optional: Clean up dummy image
    # if 'dummy_pil_img' in locals() and sample_image_path.exists():
    #    os.remove(sample_image_path)
    #    print(f"Cleaned up dummy image: {sample_image_path}")

    print("\n--- Inference API Example Finished ---")

--- Running Inference API Example ---
CHECKPOINT_PATH: /workspace/indic-clip/models/checkpoints


2025-04-22 13:51:18 - indic_clip.data.tokenization - INFO - Successfully loaded tokenizer: /workspace/indic-clip/models/tokenizer
2025-04-22 13:51:18 - indic_clip.data.tokenization - INFO - Custom special tokens already exist or none were specified.
2025-04-22 13:51:18 - indic_clip.data.tokenization - INFO - Tokenizer state loaded successfully from /workspace/indic-clip/models/tokenizer
2025-04-22 13:51:18 - __main__ - INFO - Loading model onto device: cuda
2025-04-22 13:51:18 - indic_clip.model.vision - INFO - Loaded timm model: resnet50 with pretrained=False
2025-04-22 13:51:18 - indic_clip.model.vision - INFO - Backbone feature dimension: 2048
2025-04-22 13:51:19 - indic_clip.model.text - INFO - Loading text model: ai4bharat/indic-bert with pretrained=False
2025-04-22 13:51:19 - indic_clip.model.text - INFO - Model hidden dimension: 768
2025-04-22 13:51:19 - indic_clip.model.text - WARNING - Initializing text model ai4bharat/indic-bert from scratch (pretrained=False).
2025-04-22 13:

Model and tokenizer loaded successfully.

Extracting image features...
Image features shape: torch.Size([1, 512])

Extracting text features...
Text features shape: torch.Size([3, 512])

Computing similarity...
Similarity matrix shape: torch.Size([1, 3])
Similarity scores:
[[ 3.089652    0.62385345 -0.25372863]]

Best matching text for the image: 'एक बिल्ली सोफे पर सो रही है।' (Score: 3.0897)

--- Inference API Example Finished ---


In [ ]:
#| hide
import nbdev
nbdev.nbdev_export() # Run this in terminal to export